<a href="https://colab.research.google.com/github/drusho/webscrape_youtube/blob/main/notebooks/2021_07_20_webscrapping_youtube_cleaning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Analyzing Youtube Tech Channels
> Using Data Collected from Selenium

- toc: false
- badges: false
- comments: true
- categories: [Selenium, Web Scrapping, Pandas]
- image: "images/thumbnails/header_youtube_web.png"

<br>

> Note: __Notebook Created by David Rusho__
* [Github Blog](https://drusho.github.io) | [Github](https://github.com/drusho/webscrape_youtube) | [Tableau](https://public.tableau.com/app/profile/drusho/) | [Linkedin](https://linkedin.com/in/davidrusho)


<br> 
> Important: This notebook contains hidden cells when viewed as a blog post.  Visit the links below for more detailed code:  
* [Google Colab Notebook](https://colab.research.google.com/drive/1UxpBBsypGqUj7816zyvGNhJcPfaxBP_c?usp=sharing): **All code** related to data cleaning and data analaysis.
* [Github - Youtube Webscrapping](https://github.com/drusho/webscrape_youtube/tree/main/code):  **All code** related to web scrapping and data collection.


## About the Data

Web scraping was performed on the _Top 10 Tech Channels_ on Youtube using _[Selenium](https://selenium-python.readthedocs.io/)_ (an automated browser (driver) controlled using python, which is often used in web scraping and web testing).  These channels were selected using a __[Top 10 Tech Youtubers](https://blog.bit.ai/top-tech-youtubers/)__ list from blog.bit.ai.  

Data from 2,000 videos was scrapped, which equals about 200 of most popular videos per channel.

## Introduction

## Data Cleaning

In [ ]:
import pandas as pd

### Import Channel Data
Data from Youtube Channels' main pages (Video and About)

In [ ]:
yt = pd.read_csv('yt_channel_scrap.csv',parse_dates=['channel_join_date'])
yt.head(2)

,Unnamed: 0,channel_name,subscribers,title,views,post_date,url,channel_join_date,channel_views,channel_description
0,0,iJustine,6.89M subscribers,Black Eyed Peas - I gotta Feeling (Parody),18M views,11 years ago,https://www.youtube.com/watch?v=iPgaTmsYTT8,NaT,NaN,NaN
1,1,iJustine,6.89M subscribers,Cake Decorating Challenge with Ro | Nerdy Numm...,12M views,5 years ago,https://www.youtube.com/watch?v=y7xZ-kJDgvM,NaT,NaN,NaN


In [ ]:
# create df of Channel details
channel_details = yt[yt.channel_join_date.notna()]
channel_details = channel_details.drop(columns=['Unnamed: 0','subscribers','title','views','post_date']).reset_index(drop=True)
channel_details.head(2)

,channel_name,url,channel_join_date,channel_views,channel_description
0,iJustine,NaN,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl..."
1,Android Authority,NaN,2011-04-03,"767,860,795 views","Your source for the best phones, streaming, ap..."


In [ ]:
#create df Video details
video_details = yt[yt.channel_join_date.isna()]
video_details = video_details.drop(columns=['Unnamed: 0','channel_join_date','channel_views','channel_description','post_date']).reset_index(drop=True)
video_details.head(2)

,channel_name,subscribers,title,views,url
0,iJustine,6.89M subscribers,Black Eyed Peas - I gotta Feeling (Parody),18M views,https://www.youtube.com/watch?v=iPgaTmsYTT8
1,iJustine,6.89M subscribers,Cake Decorating Challenge with Ro | Nerdy Numm...,12M views,https://www.youtube.com/watch?v=y7xZ-kJDgvM


### Merge Channel Related Dataframes

In [ ]:
# merge dfs 
merged = channel_details.merge(video_details, on='channel_name')
merged.head(2)

,channel_name,url_x,channel_join_date,channel_views,channel_description,subscribers,title,views,url_y
0,iJustine,NaN,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,Black Eyed Peas - I gotta Feeling (Parody),18M views,https://www.youtube.com/watch?v=iPgaTmsYTT8
1,iJustine,NaN,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,Cake Decorating Challenge with Ro | Nerdy Numm...,12M views,https://www.youtube.com/watch?v=y7xZ-kJDgvM


In [ ]:
# drop 2nd url column and rename remaining url col
merged.drop(columns=('url_x'),inplace=True)
merged.rename(columns={'url_y':'url'},inplace=True)
merged.head()

,channel_name,channel_join_date,channel_views,channel_description,subscribers,title,views,url
0,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,Black Eyed Peas - I gotta Feeling (Parody),18M views,https://www.youtube.com/watch?v=iPgaTmsYTT8
1,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,Cake Decorating Challenge with Ro | Nerdy Numm...,12M views,https://www.youtube.com/watch?v=y7xZ-kJDgvM
2,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,The Voice of Siri!,11M views,https://www.youtube.com/watch?v=W2bc72HClEE
3,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,Ugliest iPhone Cases Ever?,9.4M views,https://www.youtube.com/watch?v=x06yBIHu26o
4,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6.89M subscribers,Making a mini cake with Ro!,9.1M views,https://www.youtube.com/watch?v=MdmGtxyzwHA


In [ ]:
# dtypes to float for views and subscribers
merged.subscribers = merged.subscribers.str.replace('M subscribers','').astype('float')*1000000
merged.head(2)

,channel_name,channel_join_date,channel_views,channel_description,subscribers,title,views,url
0,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6890000.0,Black Eyed Peas - I gotta Feeling (Parody),18M views,https://www.youtube.com/watch?v=iPgaTmsYTT8
1,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6890000.0,Cake Decorating Challenge with Ro | Nerdy Numm...,12M views,https://www.youtube.com/watch?v=y7xZ-kJDgvM


In [ ]:
# modify views col dtype to float
def fix_views(col):
  if 'M' in col:
    return float(col.replace('M views',''))*1000000
  elif 'K' in col:
    return float(col.replace('K views',''))*1000
  elif '1 year ago' in col:
    return 0

merged['views'] = merged['views'].apply(fix_views)

merged.head(2)

,channel_name,channel_join_date,channel_views,channel_description,subscribers,title,views,url
0,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6890000.0,Black Eyed Peas - I gotta Feeling (Parody),18000000.0,https://www.youtube.com/watch?v=iPgaTmsYTT8
1,iJustine,2006-05-07,"1,288,987,476 views","Tech, video games, failed cooking attempts, vl...",6890000.0,Cake Decorating Challenge with Ro | Nerdy Numm...,12000000.0,https://www.youtube.com/watch?v=y7xZ-kJDgvM


In [ ]:
# Correct channel view column to display num only
merged['channel_views'] = merged['channel_views'].str.replace(',','').str.replace(' views','').astype('int')

### Import Videos Data

Specific data from 2000 youtube videos

In [ ]:
# import videos 
df_videos = pd.read_csv('yt_videos_scrap_big_data.csv',parse_dates=['Publish Date','Upload_date'])
df_videos.drop(columns=['Unnamed: 0','Duration','Channel Name','Title'],inplace=True)
df_videos.sample(2)

,url,Partial Description,Publish Date,Upload_date,Genre,Width,Height,Likes,Comments,Interaction Count
1186,https://www.youtube.com/watch?v=P0r9wR-Z2dc,For $200 how does a new vs used Windows 10 lap...,2018-03-11,2018-03-11,Science & Technology,1280.0,720.0,31K,"3,469 Comments",1634850
1209,https://www.youtube.com/watch?v=j6T1Mygucak,Use sharp scissors like these - http://amzn.to...,2012-12-08,2012-12-08,Science & Technology,1280.0,720.0,46K,"3,257 Comments",14169813


In [ ]:
# comments dytpe to int
df_videos['Comments'] = df_videos['Comments'].str.replace('Comments','').str.replace(',','').astype('int')
df_videos.sample(2)

,url,Partial Description,Publish Date,Upload_date,Genre,Width,Height,Likes,Comments,Interaction Count
1087,https://www.youtube.com/watch?v=TOyazdH2b-U,That's it. Ken has officially gone too far in ...,2019-04-01,2019-04-01,Science & Technology,1280.0,720.0,55K,2890,2348400
1744,https://www.youtube.com/watch?v=K43mTKyaed8,It’s that time again! We’ve got another massiv...,2017-10-14,2017-10-14,Science & Technology,1280.0,720.0,18K,1397,769452


In [ ]:
# modify likes col dtype to float
def fix_likes(col):
  if 'M' in col:
    return float(col.replace('M',''))*1000000
  elif 'K' in col:
    return float(col.replace('K',''))*1000
  else:
    return float(col)

df_videos['Likes'] = df_videos['Likes'].apply(fix_likes)

df_videos.head(2)

,url,Partial Description,Publish Date,Upload_date,Genre,Width,Height,Likes,Comments,Interaction Count
0,https://www.youtube.com/watch?v=iPgaTmsYTT8,Thanks for watching! Don't forget to subscribe...,2009-07-30,2009-07-30,Comedy,1280.0,720.0,102000.0,23437,18198670
1,https://www.youtube.com/watch?v=y7xZ-kJDgvM,Thanks for watching! Don't forget to subscribe...,2016-02-18,2016-02-18,Howto & Style,1280.0,720.0,99000.0,8421,12395700


In [ ]:
# Fix Width and Height, remove '.' and '0' from end of str
df_videos['Width'] = df_videos['Width'].astype('str').str.split(".", expand=True)[0]
df_videos['Height'] = df_videos['Height'].astype('str').str.split(".", expand=True)[0]
df_videos.head(2)

,url,Partial Description,Publish Date,Upload_date,Genre,Width,Height,Likes,Comments,Interaction Count
0,https://www.youtube.com/watch?v=iPgaTmsYTT8,Thanks for watching! Don't forget to subscribe...,2009-07-30,2009-07-30,Comedy,1280,720,102000.0,23437,18198670
1,https://www.youtube.com/watch?v=y7xZ-kJDgvM,Thanks for watching! Don't forget to subscribe...,2016-02-18,2016-02-18,Howto & Style,1280,720,99000.0,8421,12395700


### Merging All Dataframes

In [ ]:
vc_merged = merged.merge(df_videos, on='url') 

# rename columns to increase readability in analysis plots and tables
vc_merged.rename(columns={
    'channel_name':'Channel Name',
    'channel_join_date':'Channel Join Date',
    'channel_views':'Channel Views (M)',
    'subscribers':'Subscribers (M)',
    'Interaction Count':'Interactations (M)',
    'views':'Video Views (M)',
    'Partial Description':'Video Desc',
    'Publish Date':'Publish Date',
    'Upload_date':'Upload Date',
    'Genre':'Video Genre',
    'Width':'Width',
    'Height':'Height',
    'Comments':'Video Comments',
    'title':'Video Title',
    'url':'Video URL'
    },inplace=True)

vc_merged.head(2)

,Channel Name,Channel Join Date,Channel Views (M),channel_description,Subscribers (M),Video Title,Video Views (M),Video URL,Video Desc,Publish Date,Upload Date,Video Genre,Width,Height,Likes,Video Comments,Interactations (M)
0,iJustine,2006-05-07,1288987476,"Tech, video games, failed cooking attempts, vl...",6890000.0,Black Eyed Peas - I gotta Feeling (Parody),18000000.0,https://www.youtube.com/watch?v=iPgaTmsYTT8,Thanks for watching! Don't forget to subscribe...,2009-07-30,2009-07-30,Comedy,1280,720,102000.0,23437,18198670
1,iJustine,2006-05-07,1288987476,"Tech, video games, failed cooking attempts, vl...",6890000.0,Cake Decorating Challenge with Ro | Nerdy Numm...,12000000.0,https://www.youtube.com/watch?v=y7xZ-kJDgvM,Thanks for watching! Don't forget to subscribe...,2016-02-18,2016-02-18,Howto & Style,1280,720,99000.0,8421,12395700


## Data Cleaning Complete

Sample of fully cleaned and merged dataframe

Data from Youtubes Channels and all Videos pages merged.

In [ ]:
# shorten column numbers length by millions 

vc_merged['Channel Views (M)'] = round(vc_merged['Channel Views (M)']/1000000,2)
vc_merged['Video Views (M)'] = vc_merged['Video Views (M)']/1000000
vc_merged['Subscribers (M)'] = vc_merged['Subscribers (M)']/1000000
vc_merged['Interactations (M)'] = round(vc_merged['Interactations (M)']/1000000,2)

vc_merged.head(2)

,Channel Name,Channel Join Date,Channel Views (M),channel_description,Subscribers (M),Video Title,Video Views (M),Video URL,Video Desc,Publish Date,Upload Date,Video Genre,Width,Height,Likes,Video Comments,Interactations (M)
0,iJustine,2006-05-07,1288.99,"Tech, video games, failed cooking attempts, vl...",6.89,Black Eyed Peas - I gotta Feeling (Parody),18.0,https://www.youtube.com/watch?v=iPgaTmsYTT8,Thanks for watching! Don't forget to subscribe...,2009-07-30,2009-07-30,Comedy,1280,720,102000.0,23437,18.2
1,iJustine,2006-05-07,1288.99,"Tech, video games, failed cooking attempts, vl...",6.89,Cake Decorating Challenge with Ro | Nerdy Numm...,12.0,https://www.youtube.com/watch?v=y7xZ-kJDgvM,Thanks for watching! Don't forget to subscribe...,2016-02-18,2016-02-18,Howto & Style,1280,720,99000.0,8421,12.4
